In [1]:
import turicreate as tc
import numpy as np

In [2]:
tc.__version__

'6.4.1'

In [3]:
from pymongo import MongoClient

In [4]:
client = MongoClient(serverSelectionTimeoutMS=50)
db = client.turidatabase

In [5]:
db.labeledinstances.find({'dsid': 4})

In [6]:
db

Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True, serverselectiontimeoutms=50), 'turidatabase')

In [33]:
def get_dataset_data(dsid):
    features = []
    labels = []
    
    for a in db.labeledinstances.find({"dsid": dsid}):
        features.append([float(val) for val in a['feature']])
        labels.append(a['label'])
        
    data = {'target': labels, 'sequence':np.array(features)}
    
    return data


def get_dataset_sframe(dsid):
    data = get_dataset_data(dsid)
    return tc.SFrame(data=data)


In [34]:
dsid_4_sframe = get_dataset_sframe(4)

## Create a defualt model

In [35]:
moel = tc.classifier.create(dsid_4_sframe, target='target')

PROGRESS: Creating a validation set from 5 percent of training data. This may take a while.
          You can set ``validation_set=None`` to disable validation tracking.

PROGRESS: The following methods are available for this type of problem.
PROGRESS: BoostedTreesClassifier, RandomForestClassifier, DecisionTreeClassifier, LogisticClassifier
PROGRESS: The returned model will be chosen according to validation accuracy.


Boosted trees classifier:

--------------------------------------------------------

Number of examples          : 265

Number of classes           : 9

...

| 3         | 6        | 1.000000  | 0.091093     | 0.701887          | 0.142857            |

| 4         | 7        | 1.000000  | 0.110862     | 0.728302          | 0.285714            |

| 9         | 13       | 0.650402  | 0.218187     | 0.803774          | 0.285714            |

+-----------+----------+-----------+--------------+-------------------+---------------------+

PROGRESS: Model selection based on validation accuracy:
PROGRESS: ---------------------------------------------
PROGRESS: BoostedTreesClassifier          : 0.5714285714285714
PROGRESS: RandomForestClassifier          : 0.5714285714285714
PROGRESS: DecisionTreeClassifier          : 0.5714285714285714
PROGRESS: LogisticClassifier              : 0.2857142857142857
PROGRESS: ---------------------------------------------
PROGRESS: Selecting BoostedTreesClassifier based on validation set performance.


In [60]:
from sklearn.decomposition import PCA

data = get_dataset_data(4)
X = data['sequence']
pca = PCA(n_components=100)
pca.fit(X) # fit data and then transform it
X_pca = pca.transform(X)

data['sequence'] = X_pca
s_frame = tc.SFrame(data=data)

moel = tc.classifier.create(s_frame, target='target')

PROGRESS: Creating a validation set from 5 percent of training data. This may take a while.
          You can set ``validation_set=None`` to disable validation tracking.

PROGRESS: The following methods are available for this type of problem.
PROGRESS: BoostedTreesClassifier, RandomForestClassifier, DecisionTreeClassifier, LogisticClassifier
PROGRESS: The returned model will be chosen according to validation accuracy.


Boosted trees classifier:

--------------------------------------------------------

Number of examples          : 265

Number of classes           : 9

...

| 3         | 13       | 1.453139  | 0.061973     | 0.947170          | 0.357143            |

| 4         | 14       | 1.453139  | 0.069339     | 0.977358          | 0.428571            |

| 9         | 20       | 1.195357  | 0.110549     | 1.000000          | 0.571429            |

+-----------+----------+-----------+--------------+-------------------+---------------------+

PROGRESS: Model selection based on validation accuracy:
PROGRESS: ---------------------------------------------
PROGRESS: BoostedTreesClassifier          : 0.42857142857142855
PROGRESS: RandomForestClassifier          : 0.5
PROGRESS: DecisionTreeClassifier          : 0.42857142857142855
PROGRESS: LogisticClassifier              : 0.5714285714285714
PROGRESS: ---------------------------------------------
PROGRESS: Selecting LogisticClassifier based on validation set performance.


In [48]:
db

Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True, serverselectiontimeoutms=50), 'turidatabase')

In [67]:
db.trainedmodels.update({"dsid": 1}, {"$set": {"ds_id":1, "ACC:": 0.99, "path": 'path13'}}, upsert=True)

/Users/nicholaslarsen/opt/anaconda3/envs/mlenv2020/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: update is deprecated. Use replace_one, update_one or update_many instead.
  """Entry point for launching an IPython kernel.


{'n': 1, 'nModified': 1, 'ok': 1.0, 'updatedExisting': True}